<a href="https://colab.research.google.com/github/Kwonjihan/ML-teamproject/blob/develop/HyunjinNoh/BERT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/ML/TP/6000 IMDB Dataset.csv'

Mounted at /content/drive


In [ ]:
import pandas as pd #dataframe 관한 함수 사용

review_df = pd.read_csv(path)
review_df #dataframe 미리보기

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,"Petter Mattei's ""Love in the Time of Money"" is...",positive
4,"Probably my all-time favorite movie, a story o...",positive
...,...,...
5995,Something somewhere must have terribly gone wr...,negative
5996,This was the next to last film appearance by J...,negative
5997,I give this movie a 4 cause I'm a die hard fan...,negative
5998,"Are we serious??? I mean wow ... just, wow. I ...",negative


In [ ]:
#import pandas as pd #dataframe 관한 함수 사용
from sklearn.model_selection import train_test_split #train_test_split 함수 사용

feature = review_df['review'].tolist() #review_df의 review 열을 list로 만들어서 feature로 정의
label = review_df['sentiment'].tolist() #review_df의 sentiment 열을 list로 만들어서 label로 정의

#positive label을 1로 인코딩
for i in range(len(label)):
    if label[i] == 'positive':
        label[i] = 1
    else :
        label[i] = 0

#train_test_split 함수 https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
#sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)[source]
train_texts, test_texts, train_labels, test_labels = train_test_split(feature, label, train_size=0.7, random_state=42) #test_size는 0.3비율로 자동으로 설정됨, random seed 값은 임의로 42로 설정

실험해볼 수 있는 것: random seed 다르게

In [58]:
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import BertTokenizer
import torch

# 텍스트 토큰화
#bert-base-uncased 모델은 pretrain된 BERT 토크나이저 모델인데, 모든 텍스트를 소문자로 만들어준다음 토큰화시킨다.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #cased 로 실험해보기

#인코딩 시 필요한 max_length 결정=>600까지 해본 결과: 512로 설정
'''
token_lens = []
for text in feature:
  tokens = tokenizer.encode(text, truncation=True) #truncation은 최대길이보다 긴 시퀀스(문장이나 문단을 칭함)를 잘려지도록 함
  token_lens.append(len(tokens)) #각 텍스트를 토큰화했을 때 생성된 토큰의 개수를 배열에 저장
sns.displot(token_lens)
plt.xlim([0, 512]);
plt.xlabel('Token count') #x축은 각 텍스트의 토큰 수, y축은 그 토큰 수를 가진 텍스트들이 얼마나 있는지
'''

#참고: BertTokenizer의 special token
'''
#[CLS]: 문장의 시작에 들어감 [SEP]: 문장의 끝에 들어감  [PAD]: padding에 들어감 [UNK]: training set 에 없던 토큰
#id값: 101, 102, 0, 100 코드: print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id, tokenizer.unk_token_id)
'''

#인코딩
train_encodings = tokenizer.batch_encode_plus( #train_texts 리스트를 처리해야 해서 batch_encode_plus
  train_texts,
  max_length=512,
  truncation=True, #truncation은 최대길이보다 긴 시퀀스(문장이나 문단을 칭함)를 잘려지도록 함
  padding='max_length', #padding은 최대길이보다 짧은 시퀀스에 패딩 토큰을 삽입하여 모든 시퀀스를 동일한 길이로 만듦
  add_special_tokens=True,
  return_attention_mask=True, #padding된 건 0으로 반환
  return_tensors='pt' #토크나이저가 반환하는 것은 pt라는 PyTorch 텐서
)

test_encodings = tokenizer.batch_encode_plus( #test_texts 리스트를 처리해야 해서 batch_encode_plus
  test_texts,
  max_length=512,
  truncation=True, #truncation은 최대길이보다 긴 시퀀스(문장이나 문단을 칭함)를 잘려지도록 함
  padding='max_length', #padding은 최대길이보다 짧은 시퀀스에 패딩 토큰을 삽입하여 모든 시퀀스를 동일한 길이로 만듦
  add_special_tokens=True,
  return_attention_mask=True, #padding된 건 0으로 반환
  return_tensors='pt' #토크나이저가 반환하는 것은 pt라는 PyTorch 텐서
)

train_encodings
#참고: train_encodings, test_encodings는 input_ids, token_type_ids, attention_mask 세 종류를 반환
'''
input_ids는 각 문장별 token들의 인코딩값
token_type_ids는 input_ids의 어느 부분이 첫번째 문장(0)이고 어느 부분이 두 번째 문장(1)인지 알려줌 #왜 다 0???
attention_mask는 padding 여부(padding 처리 되었다면 0)
'''

{'input_ids': tensor([[  101,  2023,  3185,  ...,     0,     0,     0],
        [  101,  2145,  1996,  ...,  2162,  2003,   102],
        [  101, 21660,  2066,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  2123,  ...,     0,     0,     0],
        [  101,  2023,  2003,  ...,     0,     0,     0],
        [  101,  2009,  1005,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

실험해볼 수 있는 것: 'bert-base-cased'

In [60]:
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertForSequenceClassification, AdamW
from tqdm import tqdm #진행도 시각화

# 데이터셋 텐서로 생성
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))
# 데이터로더 생성: 데이터셋들을 미니배치로 나누어 모델에 공급.
##하이퍼파라미터 batch_size: 후보 16, 32
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 모델 준비
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# 훈련 설정
##하이퍼파라미터 learning rate: 후보 5e-5, 3e-5, 2e-5
optimizer = AdamW(model.parameters(), lr=5e-5)
# epoch 횟수 설정
##하이퍼파라미터 epoch_num: 후보 2, 3, 4
epoch_num = 2

model.train()
for epoch in range(epoch_num): #epoch은 전체 훈련 데이터셋을 한 번 통과하는 것을 의미
    for batch in tqdm(train_loader):
        optimizer.zero_grad() #optimizer의 gradient를 초기화. 각 미니배치마다 새로운 그래디언트 계산 위해.

        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

# 평가
model.eval()

#변수 초기화
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        total += labels.size(0)
        correct += (predicted_labels == labels).sum().item()

accuracy = correct / total
print(f'테스트 정확도: {accuracy}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 263/263 [26:36<00:00,  6.07s/it]


테스트 정확도: 0.905


실험해볼 수 있는 것: BERT의 다른 classification 모델, BERTAdam 등 다른 optimizer

In [ ]:
from sklearn.model_selection import GridSearchCV
#Grid Search
#하이퍼파라미터는 BERT 논문 작성자가 파인튜닝으로 권장하는 것들(BERT 논문 부록 A.3) + medium post https://medium.com/distributed-computing-with-ray/hyperparameter-optimization-for-transformers-a-guide-c4e32c6c989b
param_grid = {
    'lerning_rate': [5e-5, 3e-5, 2e-5],
    'batch_size': [16, 32], #늘리면 훈련시간이 줄어들지만 정확도는 낮아짐.
    'number_of_epochs': [2, 3, 4],

    'weight_decay': [0, 0.3], #weight_decay값이 커질수록 weight로 인한 오버피팅 방지
    'warmup_steps': [0, 500]  #medium 포스트 참고
}

#참고 코드
'''
grid_cv = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5) #뒤의 문제에서 최적의 파라미터로 다시 전체 train set에 대해 학습시킬 것이기 때문에, refit=True 옵션은 굳이 쓰지 않았음
grid_cv.fit(X_train, y_train) #grid_cv 작동시킴

best_parameters = grid_cv.best_params_ #best parameter 찾음
best_score = grid_cv.best_score_ #best validation score 찾음

print(f"Best parameters: {best_parameters}")
print(f"Best score: {best_score}")
'''

실험해볼 수 있는 것: grid search 수행해보기, grid search의 발전 버전 Improving Grid Search with Bayesian Optimization도 수행해보기

